In [1]:
from openai import OpenAI
# import openai
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def get_chatbot_response(client, model_name, messages, temperature=0.0):
    input_messages = []
    for message in messages: 
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,  # no randomness desired as agent-based system will depend on each other
        top_p=0.8,
        max_tokens=2000  # word or sub-word
    ).choices[0].message.content

    return response

In [3]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # placeholder, Ollama doesn't check it
)

model_name = "phi3"

In [ ]:
messages = [
    {"role": "system", "content": "Capital of India? Single word answer."}
]

response = get_chatbot_response(client, model_name, messages)

print(response)

# Prompt Emgineering

## Technique 1: Structured output

In [37]:
system_prompt = """
You are a helpful assistant that answers questions about the capitals of countries.

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:

[
    {
        "country": the country that you will get the capital of,
        "capital": the capital of the country stated
    }
]
"""

# List of dictionaries
# Such structure is preferable so that we can put it in a database easily

messages = [
    {"role": "system", "content": system_prompt},
]

messages.append({"role": "user", "content": "What is the capital of India?"})

response = get_chatbot_response(client, model_name, messages)

print(response)

```json

[

    {

        "country": "India",

        "capital": "New Delhi"

    }

]

```


## Instruction 2 (Much more difficult with at least 5 added constraints)


In [27]:
type(response)  # string

str

In [ ]:
json_response = json.loads(response)

print(json_response)

In [ ]:
type(json_response)  # list
type(json_response[0])  # dict

json_response[0]["country"]  # "India"
json_response[0]["capital"]  # "New Delhi"

# Technique 2: Input Structuring

In [40]:
user_input = """ 
    Get me the capital of the following countries:
    ```
        1. Italy
        2. France
        3. Germany

    ```
"""

messages = [
    {"role": "system", "content": system_prompt},
]

messages.append({"role": "user", "content": user_input})

response = get_chatbot_response(client, model_name, messages)

print(response)  # list

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    }
]



In [41]:
json_response = json.loads(response)

print(json_response)

[{'country': 'Italy', 'capital': 'Rome'}, {'country': 'France', 'capital': 'Paris'}, {'country': 'Germany', 'capital': 'Berlin'}]


# Technique 3: Chain of Thought

In [ ]:
user_prompt = """
    Calculate the result of this equation: 1+3
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

# But making the equation complex will cause the model to fail and give incorrect result
# We can make the accuracy better by telling it to do the calculation step by step


```json

{

    "result": 4

}

```


In [ ]:
(1234*43)/567-123+94567

94537.58377425044

In [ ]:
user_prompt = """
    Calculate the result of this equation: (1234*43)/567-123+94567
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)


```json

{

    "result": -123456789.0

}

```


In [ ]:
# Taking the code again

user_prompt = """
    Calculate the result of this equation: 1234*43/567-123+94567
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "steps: This is where you solve the equation bit by bit following  the BEDMAS rule. You need to show your work and calculate each step leading to the final result. Feel free to write in free text",
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)


```json

{

    "steps": [

        {

            "step_description": "First, perform multiplication and division as per BEDMAS rule.",

            "calculation": "(1234*43)/567",

            "result": 9.085

        },

        {

            "step_description": "Next, subtract the result from step one from 123 and then add 94567.",

            "calculation": "(123 - (9.085)) + 94567",

            "result": 94685.915

        }

    ],

    "result": 94685.915

}

```


# RAG

In [ ]:
user_prompt = """
    What's new in iPhone 16?
"""

# Our LLM (Phi-3) does not know the existance of iPhone 16
# It was released after the model was trained

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

As of my knowledge cutoff date, the iPhone 16 has not been released. However, if we speculate based on Apple’s typical pattern and recent trends up to early 2024:

- **Display**: The display might have a higher refresh rate for smoother visuals in gaming or video playback; possibly an OLED screen with improved color accuracy.

- **Camera System**: Apple could introduce new camera features like advanced night mode, enhanced portrait lighting effects, and AI improvements to optimize photo quality automatically.

- **Performance**: The processor might be more powerful than the current generation's A17 chip with improved efficiency for better battery life or augmented reality (AR) capabilities.

- **Design Changes**: Potential design updates could include a new color option, an updated notch size if still present, and possibly water resistance improvements.

- **Software Updates**: iOS 16 might introduce significant privacy features like App Library for better app management or enhancement

In [44]:
iPhone_16 = """
The iPhone 16 lineup brings notable upgrades: faster and smarter processing (A18 chip), expanded AI readiness, advanced camera features including a dedicated camera control button, improved battery and safety tools, and better customization with Photographic Styles. Whether you go for the standard or Pro version, Apple’s renewed focus on on-device intelligence and user control is its biggest leap since the iPhone X era.
"""

In [ ]:
user_prompt = f"""
    {iPhone_16}

    What's new in iPhone 16?
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

The latest iteration of iPhones introduces several significant enhancements: The A18 chip now offers unparalleled processing speed, enabling smoother multitasking and more responsive gaming. Apple has also expanded the capabilities for on-device artificial intelligence (AI), allowing apps to offer personalized experiences without constant internet connectivity or data sharing.


The camera system receives a substantial upgrade with an additional dedicated control button that lets users easily switch between cameras, adjust settings like exposure and focus directly from the screen, and access new features such as Night Sight for low-light photography. The battery life has been improved to support longer usage times on both iOS and iPadOS devices without compromising performance or safety measures introduced in previous models.


Customization options have also expanded with more granular control over Photographic Styles, enabling users to tailor the look of their photos even further acc

## Automatically extract context data from Database

In [42]:
samsung_galaxy_s23 = """
Key Specifications (Galaxy S23)

    Display & Build: 6.1‑inch Dynamic AMOLED 2X, Full HD+ (2340 × 1080 px), 120 Hz refresh rate with Gorilla Glass Victus 2 protection and IP68 dust/water resistance
    .
    Processor: 4 nm Snapdragon 8 Gen 2 (for Galaxy), paired with Adreno 740 GPU
    .
    Memory & Storage: Comes with 8 GB LPDDR5X RAM; storage options include 128 GB (UFS 3.1), 256 GB, or 512 GB (UFS 4.0)
    .

    Cameras: Rear triple setup – 50 MP wide (OIS), 12 MP ultrawide, and 10 MP telephoto (3× optical zoom); front camera is 12 MP
    .

    Battery & Charging: 3,900 mAh battery with 25 W fast charging (0–50% in ~30 mins), also supports 10 W wireless and 4.5 W reverse wireless charging
    .

    Software & Extras: Ships with Android 13 + One UI 5.1; includes in-display ultrasonic fingerprint reader, Gorilla Glass Victus 2, IP68 rating, DeX support, Samsung Pay, and more
    .

India Pricing (Launch & Current Deals)

    Launch prices (India):

        8 GB + 128 GB: ₹74,999

        8 GB + 256 GB: ₹79,999
        mint
        GSMArena
        The Times of India
        .

    Current market prices (as of July 2025):

        Starting price has dropped to around ₹44,999 for the base variant and ₹49,999 for 256 GB version
        Gadgets 360
        .

    Flash Sale Example:

        On Flipkart, a 256 GB variant was offered at just ₹42,999 (55% off), including additional bank card cashback benefits
        Reddit
        .

Background & Release Highlights

    The Galaxy S23 series was officially unveiled at Samsung’s Unpacked event (Feb 1, 2023) and launched globally on February 17, 2023
    Lifewire
    WIRED
    .

    All models in the S23 lineup (S23, S23+, S23 Ultra) are powered by Snapdragon 8 Gen 2, feature HDR10+, wireless charging, and Gorilla Glass Victus 2
"""

In [45]:
data = [iPhone_16, samsung_galaxy_s23]

In [46]:
user_prompt = """What's new in iPhone 16?"""

## Local embedding endpoint

In [77]:
from sentence_transformers import SentenceTransformer

embedding_model_name = 'all-MiniLM-L6-v2'

model = SentenceTransformer(embedding_model_name)

In [ ]:
def get_embedding_vector(prompt, embedding_model_name):
    embedding_vector = model.encode(prompt)

    embedding = []

    for embedding_obj in embedding_vector:
        embedding.append(embedding_obj)

    return embedding

In [87]:
user_prompt

"What's new in iPhone 16?"

In [88]:
user_prompt_embedding = get_embedding_vector(user_prompt, model)

user_prompt_embedding[:10]

[-0.049540382,
 0.034526207,
 0.029998489,
 -0.051713217,
 0.015609871,
 -0.037609097,
 0.015719926,
 -0.020039653,
 -0.015138678,
 0.030683188]

In [94]:
print(type(user_prompt_embedding))
print(len(user_prompt_embedding))
print(embedding_model_name)

<class 'list'>
384
all-MiniLM-L6-v2


In [98]:
data

['\nThe iPhone 16 lineup brings notable upgrades: faster and smarter processing (A18 chip), expanded AI readiness, advanced camera features including a dedicated camera control button, improved battery and safety tools, and better customization with Photographic Styles. Whether you go for the standard or Pro version, Apple’s renewed focus on on-device intelligence and user control is its biggest leap since the iPhone\u202fX era.\n',
 '\nKey Specifications (Galaxy S23)\n\n    Display & Build: 6.1‑inch Dynamic\u202fAMOLED\u202f2X, Full\u202fHD+ (2340\u202f×\u202f1080\u202fpx), 120\u202fHz refresh rate with Gorilla\u202fGlass\u202fVictus\u202f2 protection and IP68 dust/water resistance\n    .\n    Processor: 4 nm Snapdragon 8 Gen 2 (for Galaxy), paired with Adreno 740 GPU\n    .\n    Memory & Storage: Comes with 8\u202fGB LPDDR5X RAM; storage options include 128\u202fGB (UFS 3.1), 256\u202fGB, or 512\u202fGB (UFS 4.0)\n    .\n\n    Cameras: Rear triple setup – 50\u202fMP wide (OIS), 12\u2

In [101]:
type(data)

list

In [107]:
data_embeddings = [get_embedding_vector(text, model) for text in data]

data_embeddings[0][:10]

[-0.0064492584,
 0.057807885,
 0.07686596,
 -0.083998404,
 0.042045616,
 -0.0058477535,
 -0.00687845,
 0.038082894,
 -0.023061134,
 0.06327745]

In [108]:
print(type(data_embeddings))
print(len(data_embeddings))
print(embedding_model_name)

<class 'list'>
2
all-MiniLM-L6-v2


In [111]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

data_similarity

array([[0.70215064, 0.35572684]], dtype=float32)

In [113]:
closest_entry_index = data_similarity.argmax()

closest_entry_index

0

In [114]:
data[closest_entry_index]

'\nThe iPhone 16 lineup brings notable upgrades: faster and smarter processing (A18 chip), expanded AI readiness, advanced camera features including a dedicated camera control button, improved battery and safety tools, and better customization with Photographic Styles. Whether you go for the standard or Pro version, Apple’s renewed focus on on-device intelligence and user control is its biggest leap since the iPhone\u202fX era.\n'

In [ ]:
# First one in data is iPhone 16 and second one is Galaxy S23
# User prompt is "What's new in iPhone 16?"
# We can see that the similarity is 0.7 and 0.36 for iPhone 16 and Galaxy S23 respectively
# This means that the iPhone 16 is more similar to the user prompt than the Galaxy S23 - as expected

In [115]:
user_prompt_with_data = f"""
{data[closest_entry_index]}
{user_prompt}
""" 

print(user_prompt_with_data)



The iPhone 16 lineup brings notable upgrades: faster and smarter processing (A18 chip), expanded AI readiness, advanced camera features including a dedicated camera control button, improved battery and safety tools, and better customization with Photographic Styles. Whether you go for the standard or Pro version, Apple’s renewed focus on on-device intelligence and user control is its biggest leap since the iPhone X era.

What's new in iPhone 16?



In [116]:
messages = [{"role": "user", "content": user_prompt_with_data}]

response = get_chatbot_response(client, model_name, messages)

print(response)


The latest iteration of iPhones introduces several significant enhancements that cater to both performance enthusiasts and those who prioritize safety, customization, and battery life:

- **A18 Chip**: The A18 chip is a powerhouse in processing speed and efficiency. It's designed for faster multitasking and seamless operation of advanced apps that require real-time data analysis like augmented reality (AR) experiences or high-definition video editing tools, which are becoming increasingly popular among users who enjoy creative software on their smartphones.

- **Expanded AI Readiness**: With the new chip and improved machine learning algorithms, iPhones can now better understand user preferences through voice commands and predictive text input without needing to connect to Wi-Fi or cellular data constantly. This is particularly beneficial for users who rely on their phones as a primary tool in professional settings like journalism where quick access to information while maintaining pr